In [ ]:
from datetime import datetime
import time
import pandas as pd
import pickle
from ib_insync import *
import nest_asyncio
import logging,sys
from k_producer import *
from util_redis import *



def histBar(contract_d,param_d,useRTH=False):
    try:
        print('starting')
        df = pd.DataFrame()
        contract = Stock(contract_d['symbol'],contract_d['exchange'], contract_d['currency'])
        bars = ib.reqHistoricalData(contract,endDateTime=param_d['endt'], durationStr=param_d['dur'],barSizeSetting=param_d['candles'], whatToShow=param_d['show'], useRTH=useRTH)
        df = util.df(bars)
        
        df = df.set_index('date')
#         df.index.tz_localize('America/Toronto', nonexistent='shift_forward')
        df.columns = map(str.lower, df.columns)
#         display(df.reset_index().dtypes)
        display(len(df))

    except Exception as e: print(e)
    finally:
        print('ended')
        return df
    
if __name__ == '__main__':
    logging.getLogger("asyncio").setLevel(logging.INFO)
    logging.basicConfig(
    level=logging.INFO,
    format='%(threadName)10s %(name)18s: %(message)s',stream=sys.stderr,)

    nest_asyncio.apply()
    
    util.startLoop()
    ib = IB()
    ib.connect('host.docker.internal', 7497, clientId=15)
    

    try:
        while True:
            time.sleep(1)
            item = lpop('hist.queue')
            if item !=None: 
                data = histBar(item['contract'],item['params'])
                if len(data)!=0:
                    p = pickle.dumps(data)
#                     with open('t1d.pickle', 'wb') as f:
#                         pickle.dump(data,f)
#                     itemr = {**item,**{'pdata':p}}
#                     print(itemr)
                    ProduceB('tester',item,p)
    except Exception as e: 
        print("FATAL: ",e)
    finally:ib.disconnect()

MainThread   ib_insync.client: Connecting to host.docker.internal:7497 with clientId 15...
MainThread   ib_insync.client: Connected
MainThread   ib_insync.client: Logged on to server version 152
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfuture
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfarm.nj
MainThread  ib_insync.wrapper: Warning 2104, reqId -1: Market data farm connection is OK:usfarm
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:euhmds
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:ushmds.nj
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:fundfarm
MainThread  ib_insync.wrapper: Warning 2106, reqId -1: HMDS data farm connection is OK:ushmds
MainThread  ib_insync.wrapper: Warning 2158, reqId -1: Sec-def data farm connection is OK:secdefil
MainThread   ib_insync.client: A

starting


3740

ended
starting


3840

...

ended
starting


192

ended
starting


192

ended
